In [18]:
__import__('pysqlite3')
import pysqlite3
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

In [2]:
# load .env OPENAI_API_KEY
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=500)

In [4]:
# load PDF
pdf_path = "DOC-SF238339076816-20230503.pdf"

loader = PyPDFLoader(pdf_path, extract_images=False)

pages = loader.load_and_split()

In [5]:
len(pages)

31

In [6]:
# Splitters
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

parent_splitter = RecursiveCharacterTextSplitter(
  chunk_size=4000,
  chunk_overlap=200,
  length_function=len,
  add_start_index=True
)

In [10]:
# Storages

# Source - https://stackoverflow.com/a
# Posted by pintz
# Retrieved 2026-01-18, License - CC BY-SA 4.0

store = InMemoryStore()
vector_store = Chroma(embedding_function=embeddings, persist_directory='child_vector_db')

In [12]:
parent_document_retriever = ParentDocumentRetriever(
  vectorstore=vector_store,
  docstore=store,
  child_splitter=child_splitter,
  parent_splitter=parent_splitter
)

parent_document_retriever.add_documents(pages,ids=None)

In [13]:
parent_document_retriever.vectorstore.get()

{'ids': ['f4726c86-c616-4704-863c-58aa4e48fedc',
  '07e7f5ef-471d-481c-b7ff-1a033be31423',
  'ca22e7aa-911d-4bb9-89ed-ddfccd20893e',
  '931a08b1-03f3-456c-a128-7d1f0618c895',
  '517b32c6-11af-4e2a-9d32-59f48d5a4401',
  'ae65837e-c472-4b90-b954-55bb47565550',
  'afc8a553-9617-4485-bfd0-567124fd2b9d',
  'dea498f6-95df-4941-84d1-09efc5d59bfc',
  '0bd6e24e-e486-4017-b167-7e1728d2b4f9',
  '6bc18b16-93fa-4031-893b-0110a5d29362',
  '78de151a-1b14-421a-a310-51d9148aad08',
  '56709c4d-133a-4b01-956b-a5c1b9c8b1b5',
  '8ff6ea18-7641-4a97-a0ac-6f8a090ff53f',
  'fe75984d-1e41-4a23-aa8c-34da11e1a62c',
  '3ca6974b-8b2c-4589-99f3-d534a7efcb1a',
  'd8af5c16-0acb-40b1-b176-0ded6a75d146',
  'c0c6108a-655c-4b9a-91e7-f4a590f8b781',
  '0a60ce7a-69ea-4815-babe-c3a9bfeba38f',
  'a9eb87af-9497-4cba-8ec4-50f6a763a574',
  'b014f325-3b29-4c18-8d26-0adaff091441',
  '084f43d3-7914-4e03-8afd-be092393d27a',
  '1d8551c3-2e16-4894-9676-e35ff56c1530',
  '39612a81-a431-410a-9ccb-6c8e9fcaa52e',
  '03350dc6-a165-4454-906f-

In [17]:
TEMPLATE = """
  Você é um especialista em legislação e tecnologia. Responda a pergunta abixo utilizando o contexto informado.
  Query:
    {question}

    Context:
    {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [19]:
setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": parent_document_retriever})

output_parser = StrOutputParser()

In [20]:
# create rag chain
# receive question -> get context from rag -> send to LLM -> parser the output
parent_chain_retrieval = setup_retrieval | rag_prompt | llm | output_parser

In [21]:
parent_chain_retrieval.invoke("Quais os principais riscos do marco legal de IA?")

'Os principais riscos do marco legal de IA podem incluir a falta de clareza nas definições e diretrizes estabelecidas, o que poderia levar a interpretações ambíguas e inconsistências na aplicação da lei. Além disso, a questão da fiscalização e cumprimento das normas também pode representar um desafio, especialmente considerando a complexidade das tecnologias de inteligência artificial e a necessidade de monitoramento constante para garantir a conformidade. Outro risco potencial é a possibilidade de limitar a inovação e o desenvolvimento tecnológico, caso as regulamentações sejam muito restritivas ou não estejam alinhadas com as práticas e padrões internacionais.'